<a href="https://colab.research.google.com/github/Sujith2k/An-Aid-for-the-Blind/blob/master/ML/Primary_model/Train/Train_yolov3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [0]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon May 18 06:59:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
 Classroom  'Colab Notebooks'	    Strings
 cohort     'component list.xlsx'   yolov3


**1) Clone the Darknet**



In [0]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 13486 (delta 2), reused 1 (delta 0), pack-reused 13478
Receiving objects: 100% (13486/13486), 12.07 MiB | 11.56 MiB/s, done.
Resolving deltas: 100% (9210/9210), done.


**2) Compile Darknet using Nvidia GPU**


In [0]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variabl

**3) Configure Darknet network for training YOLO V3**

In [0]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [0]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 18000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=6@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=6@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=6@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=33@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=33@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=33@' cfg/yolov3_training.cfg

In [0]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"
!cp cfg/yolov3_training.cfg "/mydrive/yolov3/"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [0]:
!echo "Person\nBillboard\nBus\nTraffic sign\nTruck\nCurrency" > data/obj.names
!echo -e 'classes= 6\ntrain  = darknet/data/train.txt\nvalid  = data/test.txt\nnames = darknet/data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data

mkdir: cannot create directory ‘data’: File exists


**4) Download darknet53 model**

In [0]:
# Download weights darknet model 53
#!wget https://pjreddie.com/media/files/darknet53.conv.74

In [0]:
%cd ..

/content


In [0]:
!ls 

darknet  gdrive  sample_data


In [0]:
#!cp -r 'gdrive/My Drive/cohort/Dataset/Open Images Dataset v4 (Bounding Boxes)/images' darknet/data/obj

In [0]:
import glob
images_list = []
while( len(images_list) == 0):
  images_list = glob.glob("gdrive/My Drive/cohort/Dataset/train/*.jpg")


In [0]:
print(len(images_list))
images_list[100]

17036


'gdrive/My Drive/cohort/Dataset/train/f4ca7a18d1538edb.jpg'

In [0]:
#Create training.txt file
file = open("darknet/data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close()

**6) Start the training**

In [0]:
# Start the training
#!darknet/./darknet detector train darknet/data/obj.data darknet/cfg/yolov3_training.cfg darknet/darknet53.conv.74 -dont_show        # While Training for first time
!darknet/./darknet detector train darknet/data/obj.data darknet/cfg/yolov3_training.cfg "gdrive/My Drive/yolov3/yolov3_training_last.weights" -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x